In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('digit-recognizer').getOrCreate()

train = spark.read.csv('./train.csv', header='true', inferSchema='true')
train.printSchema()


root
 |-- label: integer (nullable = true)
 |-- pixel0: integer (nullable = true)
 |-- pixel1: integer (nullable = true)
 |-- pixel2: integer (nullable = true)
 |-- pixel3: integer (nullable = true)
 |-- pixel4: integer (nullable = true)
 |-- pixel5: integer (nullable = true)
 |-- pixel6: integer (nullable = true)
 |-- pixel7: integer (nullable = true)
 |-- pixel8: integer (nullable = true)
 |-- pixel9: integer (nullable = true)
 |-- pixel10: integer (nullable = true)
 |-- pixel11: integer (nullable = true)
 |-- pixel12: integer (nullable = true)
 |-- pixel13: integer (nullable = true)
 |-- pixel14: integer (nullable = true)
 |-- pixel15: integer (nullable = true)
 |-- pixel16: integer (nullable = true)
 |-- pixel17: integer (nullable = true)
 |-- pixel18: integer (nullable = true)
 |-- pixel19: integer (nullable = true)
 |-- pixel20: integer (nullable = true)
 |-- pixel21: integer (nullable = true)
 |-- pixel22: integer (nullable = true)
 |-- pixel23: integer (nullable = true)
 |-- pi

In [2]:
from pyspark.ml.feature import PCA, VectorAssembler

cols = train.columns[:]
cols.remove('label')

assembler = VectorAssembler(inputCols=cols, outputCol="features")
train = assembler.transform(train)
pca = PCA(k=100, inputCol="features", outputCol="pcaFeatures")
model = pca.fit(train)
train = model.transform(train)

In [7]:
# train.select('pcaFeatures').show()
model.explainedVariance

DenseVector([0.0975, 0.0716, 0.0615, 0.0538, 0.0489, 0.043, 0.0328, 0.0289, 0.0277, 0.0235, 0.021, 0.0206, 0.017, 0.0169, 0.0158, 0.0148, 0.0132, 0.0128, 0.0119, 0.0115, 0.0107, 0.0102, 0.0096, 0.0091, 0.0089, 0.0084, 0.0081, 0.0078, 0.0074, 0.0069, 0.0066, 0.0064, 0.006, 0.0059, 0.0056, 0.0054, 0.0051, 0.0049, 0.0048, 0.0047, 0.0045, 0.0044, 0.0042, 0.004, 0.0038, 0.0037, 0.0036, 0.0035, 0.0034, 0.0032, 0.0032, 0.0031, 0.0029, 0.0029, 0.0028, 0.0027, 0.0027, 0.0026, 0.0025, 0.0025, 0.0024, 0.0024, 0.0023, 0.0022, 0.0021, 0.0021, 0.002, 0.002, 0.0019, 0.0019, 0.0019, 0.0018, 0.0018, 0.0017, 0.0017, 0.0016, 0.0016, 0.0015, 0.0015, 0.0014, 0.0014, 0.0014, 0.0014, 0.0014, 0.0013, 0.0013, 0.0013, 0.0012, 0.0012, 0.0012, 0.0012, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.001, 0.001, 0.001, 0.001])